<a href="https://colab.research.google.com/github/Matheusfarmaceutico/Estat-stica-e-An-lise-de-dados-com-Python/blob/main/01_Preparando_os_dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import pandas as pd
import numpy as np

In [8]:
db = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/01 - Preparação dos dados/02 - Meus arquivos/01 - Preparando o DB/microdados_enem_2019_sp.csv", sep  = ";", encoding = "iso-8859-1")

In [9]:
print(db)

       NU_INSCRICAO  CO_MUNICIPIO_RESIDENCIA NO_MUNICIPIO_RESIDENCIA  \
0          1,90E+11                  3546801            Santa Isabel   
1          1,90E+11                  3538204             Pinhalzinho   
2          1,90E+11                  3550308               São Paulo   
3          1,90E+11                  3548708   São Bernardo do Campo   
4          1,90E+11                  3549409    São Joaquim da Barra   
...             ...                      ...                     ...   
813767     1,90E+11                  3550308               São Paulo   
813768     1,90E+11                  3550308               São Paulo   
813769     1,90E+11                  3550308               São Paulo   
813770     1,90E+11                  3509205                 Cajamar   
813771     1,90E+11                  3503307                  Araras   

        CO_UF_RESIDENCIA SG_UF_RESIDENCIA  NU_IDADE TP_SEXO  TP_ESTADO_CIVIL  \
0                     35               SP        22    